In [1]:
import sys
sys.path.append('./models')
from models.BBSNet_model import BBSNetTransformerAttention
from models.BBSNet_model import BBSNetChannelSpatialAttention
from BBSNet_train import setup_gpu, get_model, get_data, main
from BBSNet_test import test
from utils import evaluate, draw_curve

%load_ext autoreload
%autoreload 2

/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from models.RFNet import RFNet
# import torch

# rgb = torch.rand((2, 3, 352, 352)).cuda()
# depth = torch.rand((2, 1, 352, 352)).cuda()
# model = RFNet(32, 50).cuda()
# l = model(rgb, depth)

# Train

In [3]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, CE, total_step)

USE GPU 0


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load data...
Start train...


/home/mahdi/Documents/pytorch/lib/python3.12/site-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


2025-08-31 04:15:44.908886 Epoch [001/002], Step [0001/2184], Loss1: 1.5685 Loss2: 1.7428
2025-08-31 04:15:58.983005 Epoch [001/002], Step [0100/2184], Loss1: 0.7616 Loss2: 0.8344
2025-08-31 04:16:13.089377 Epoch [001/002], Step [0200/2184], Loss1: 0.8836 Loss2: 0.9386
2025-08-31 04:16:27.257312 Epoch [001/002], Step [0300/2184], Loss1: 1.0934 Loss2: 1.1165
2025-08-31 04:16:41.438805 Epoch [001/002], Step [0400/2184], Loss1: 0.5544 Loss2: 0.5977
2025-08-31 04:16:55.726051 Epoch [001/002], Step [0500/2184], Loss1: 0.3452 Loss2: 0.5387
2025-08-31 04:17:10.100950 Epoch [001/002], Step [0600/2184], Loss1: 0.8891 Loss2: 0.7920
2025-08-31 04:17:24.419066 Epoch [001/002], Step [0700/2184], Loss1: 0.6258 Loss2: 0.6021
2025-08-31 04:17:38.692291 Epoch [001/002], Step [0800/2184], Loss1: 1.1424 Loss2: 1.0865
2025-08-31 04:17:53.067120 Epoch [001/002], Step [0900/2184], Loss1: 1.4821 Loss2: 1.3666
2025-08-31 04:18:07.384242 Epoch [001/002], Step [1000/2184], Loss1: 1.1774 Loss2: 0.9669
2025-08-31

# Use Trained Model

I save the model using this code:
torch.save(model.state_dict(), save_path +
                           'BBSNet_epoch_best.pth')

Now, after a long training period, I want to use it on my test data. So, how can I load this model?

In [4]:
from models.BBSNet_model import BBSNetTransformerAttention

model = BBSNetTransformerAttention()

# assume your model is called `model`
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")


Total parameters: 60028676
Trainable parameters: 60028676
Non-trainable parameters: 0


In [5]:
# test(method='bbsnet') 